In [7]:
import warnings
warnings.filterwarnings("ignore")
import torch
import torchaudio
import fairseq
import random

In [8]:
# Load the model
model_fairseq ="/home1/Amartya/outputs/2023-11-27/18-33-19/checkpoints/checkpoint_best.pt"

In [9]:
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([model_fairseq])

In [10]:
wav_input_16khz =  torchaudio.load("/home1/Amartya/sagemaker_deploy/281474982913816.wav")[0]
inp = {'source': wav_input_16khz, 'padding_mask':torch.zeros(wav_input_16khz.shape[-1])}

In [11]:
class SimpleModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = fairseq.checkpoint_utils.load_model_ensemble_and_task([model])[0]
        self.model = self.model[0].to('cpu')
        self.model.eval()

    def forward(self, x):
        # Simplify the behavior of complex_model
        with torch.no_grad():
            encoder_out = self.model(**x)
            emm = self.model.get_normalized_probs(encoder_out, log_probs=True)
        return emm.transpose(0, 1)

# Load your Wav2VecCtc model here
simple_model = SimpleModel(model_fairseq)

# Now you can trace simple_model instead of complex_model
traced_model = torch.jit.trace(simple_model, inp)

# simple_model(inp)

In [12]:
torch.jit.save(m=traced_model, f="/home1/Amartya/sagemaker_deploy/jit_models/en_jit_mdl.pt")